In [ ]:
# !pip3 install langchain
!pip install transformers
!pip install langchain[docarray]
!pip install docarray
!pip install pypdf
!pip install langchain_huggingface


Looking in indexes: https://oss.roshan-ai.ir/repository/pypi-group/simple
Looking in indexes: https://oss.roshan-ai.ir/repository/pypi-group/simple
  Using cached https://oss.roshan-ai.ir/repository/pypi-group/packages/docarray/0.32.1/docarray-0.32.1-py3-none-any.whl (215 kB)
  Attempting uninstall: docarray
    Found existing installation: docarray 0.40.0
    Uninstalling docarray-0.40.0:
      Successfully uninstalled docarray-0.40.0
Looking in indexes: https://oss.roshan-ai.ir/repository/pypi-group/simple
Fatal Python error: init_import_site: Failed to import the site module
Python runtime state: initialized
Traceback (most recent call last):
  File "/usr/lib/python3.10/site.py", line 79, in <module>
    import os
  File "/usr/lib/python3.10/os.py", line 29, in <module>
    from _collections_abc import _check_methods
  File "/usr/lib/python3.10/_collections_abc.py", line 306, in <module>
    class Reversible(Iterable):
  File "/usr/lib/python3.10/abc.py", line 106, in __new__
    cl

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline

model_name = "unsloth/gemma-2-9b-it"
model = HuggingFacePipeline.from_model_id(
    model_id=model_name,
    task="text-generation",
    device_map='cuda:0',
    model_kwargs={"load_in_4bit": True},
    pipeline_kwargs={"max_new_tokens": 256,},
)
embedding_name = 'sbunlp/fabert'
embeddings = HuggingFaceEmbeddings(
                model_name=embedding_name,
                # model_kwargs={"trust_remote_code": True},
            )


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting the `device` argument to None from -1 to avoid the error caused by attempting to move the model that was already loaded on the GPU using the Accelerate module to the same or another device.
Device set to use cuda:0
No sentence-transformers model found with name sbunlp/fabert. Creating a new one with mean pooling.
Some weights of BertModel were not initialized from the model checkpoint at sbunlp/fabert and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

def apply_chat_template_and_response(prompt):
    messages = [
    {'role': 'user', 'content': prompt}
    ]    

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False
    )

    return model.invoke(text).replace(text, '')

In [3]:
print(apply_chat_template_and_response("اسمت چیه؟"))

/home/roshan/.virtualenvs/llm/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


من Gemma هستم، یک مدل زبان بزرگ که توسط Google DeepMind آموزش دیده ام.



In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
response_from_model = apply_chat_template_and_response("Give me an inspirational quote in persian")
parsed_response = parser.parse(response_from_model)
print(parsed_response)

"هر کسی می‌تواند پرواز کند، اما فقط کسانی که دست از رویاها برنمی‌دارند."

**Translation:** "Everyone can fly, but only those who never give up on their dreams." 


This quote emphasizes the importance of perseverance and believing in oneself to achieve one's aspirations.



In [5]:
from langchain.prompts import PromptTemplate

template = """
You are a helpful and knowledgeable AI assistant. Use only the information retrieved from the documents to answer the user's question in Persian (Farsi). If the answer is not found in the retrieved context, respond with: "متاسفانه اطلاعاتی در این مورد ندارم." Do not use your own knowledge beyond the provided context. Be accurate, clear, and polite. Never mention the documents or the retrieval process in your response. Just respond naturally in Persian.
Context: {context}

Question: {question}

Answer:

"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nYou are a helpful and knowledgeable AI assistant. Use only the information retrieved from the documents to answer the user\'s question in Persian (Farsi). If the answer is not found in the retrieved context, respond with: "متاسفانه اطلاعاتی در این مورد ندارم." Do not use your own knowledge beyond the provided context. Be accurate, clear, and polite. Never mention the documents or the retrieval process in your response. Just respond naturally in Persian.\nContext: Here is some context\n\nQuestion: Here is a question\n\nAnswer:\n\n'

In [6]:
context = "من علیرضا هستم و تهران زندگی میکنم."
formatted_prompt = prompt.format(context = context, question="اسم من چیه؟")
response_from_model = apply_chat_template_and_response(formatted_prompt)
parsed_response = parser.parse(response_from_model)
print(parsed_response.replace(formatted_prompt, ""))

علیرضا 



In [7]:
formatted_prompt = prompt.format(context=context, question="من چند سالمه؟")
response_from_model = model.invoke(formatted_prompt)
parsed_response = parser.parse(response_from_model)
print(parsed_response.replace(formatted_prompt, ""))

متاسفانه اطلاعاتی در این مورد ندارم.




In [ ]:
from langchain_community.document_loaders import PyPDFLoader


loader = PyPDFLoader("...")
pages = loader.load_and_split()
#pages = loader.load()
pages

Ignoring wrong pointing object 105 0 (offset 0)


[Document(page_content='ﺑﺎﺳﻣﮫ ﺗﻌﺎﻟﯽ    اﺻﻼح ﻧﺎﻣﮫ ﺷﯾوه اﺟراﯾﯽ  ﻧﺣوه  اﯾﺟﺎد  و  ﺗرﻣﯾم  ﺳﺎﺑﻘﮫ  ﺗﺣﺻﯾﻠﯽ  ھﺎی آزﻣون  ﻧﮭﺎﯾﯽ  دوره  دوم  ﻣﺗوﺳطﮫ  ﻣﺻوب  \nﭘﻧﺟﺎه وھﻔﺗﻣﯾن)\n57\n(\n \nﺟﻠﺳﮫ\n \nﮐﻣﯾﺳﯾون\n \nﻣﻌﯾن\n \nﺷورای ﻋﺎﻟﯽ\n \nآﻣوزش\n \nو\n \nﭘرورش\n \nﺗﺎرﯾﺦ\n \n27/9/1402\n  ﺑﮫ اﺳﺗﻧﺎد  ﻣﺻوﺑﮫ  ﺟﻠﺳﮫ  1053  ﺗﺎرﯾﺦ  04/12/1403  ﻣوﺿوع  ﻋﻧﺎوﯾن  دروس  آزﻣون  ﻧﮭﺎﯾﯽ  دوره  دوم  \nﻣﺗوﺳطﮫ\n \nو\n \nﭼﮕوﻧﮕﯽ\n \nﺗرﻣﯾم\n \nﺳﺎﺑﻘﮫ\n \nﺗﺣﺻﯾﻠﯽ\n \nﺳواﺑق\n \nﺗﺣﺻﯾﻠﯽ،\n \nﺷﯾوه ﻧﺎﻣﮫ\n \nاﺟراﯾﯽ\n \nﻧﺣوه\n \nاﯾﺟﺎد\n \nﺳﺎﺑﻘﮫ\n \nﺗﺣﺻﯾﻠﯽ\n \nو\n \nﺗرﻣﯾم\n \nﺳﺎﺑﻘﮫ\n \nﺗﺣﺻﯾﻠﯽ\n \nآزﻣون ھﺎی\n \nﻧﮭﺎﯾﯽ\n \nدوره\n \nدوم\n \nﻣﺗوﺳطﮫ)اﺻﻼﺣﯾﮫ\n \nﺷﯾوه ﻧﺎﻣﮫ\n \nﻣﺻوب\n \nﭘﻧﺟﺎه\n \nو\n \nھﻔﺗﻣﯾن\n \nﺟﻠﺳﮫ\n \nﮐﻣﯾﺳﯾون\n \nﻣﻌﯾن\n \nﺷورای\n \nﻋﺎﻟﯽ\n \nآﻣوزش\n \nو\n \nﭘرورش\n \nﺗﺎرﯾﺦ\n \n27/09/1402\n(و\n \nﻣﺻوﺑﮫ\n \nﺟﻠﺳﮫ\n \n915\n \nﺷورای\n \nﻋﺎﻟﯽ\n \nاﻧﻘﻼب\n \nﻓرھﻧﮕﯽ\n \nﺗﺎرﯾﺦ.......\n \nﺑﮫ\n \nﺷرح\n \nزﯾر\n \nﺑﮫ\n \nﺗﺻوﯾب\n \nرﺳﯾد.\n ﺗﻌﺎرﯾف(اﻟف ﺳﺎﺑﻘﮫ ﺗﺣﺻﯾﻠﯽ:  ﻣطﺎﺑق  ﻣﺎده١  ﻣﺻوﺑﮫ  ﺟﻠﺳﮫ  ٣۴٨  ﺗﺎرﯾﺦ  15/04/1400  ﺷورای  ﻋﺎﻟﯽ  اﻧﻘﻼب  ﻓرھﻧﮕﯽ،  \nﺳﺎ

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
text_documents = text_splitter.split_documents(pages)[:5]

pages

[Document(page_content='ﺑﺎﺳﻣﮫ ﺗﻌﺎﻟﯽ    اﺻﻼح ﻧﺎﻣﮫ ﺷﯾوه اﺟراﯾﯽ  ﻧﺣوه  اﯾﺟﺎد  و  ﺗرﻣﯾم  ﺳﺎﺑﻘﮫ  ﺗﺣﺻﯾﻠﯽ  ھﺎی آزﻣون  ﻧﮭﺎﯾﯽ  دوره  دوم  ﻣﺗوﺳطﮫ  ﻣﺻوب  \nﭘﻧﺟﺎه وھﻔﺗﻣﯾن)\n57\n(\n \nﺟﻠﺳﮫ\n \nﮐﻣﯾﺳﯾون\n \nﻣﻌﯾن\n \nﺷورای ﻋﺎﻟﯽ\n \nآﻣوزش\n \nو\n \nﭘرورش\n \nﺗﺎرﯾﺦ\n \n27/9/1402\n  ﺑﮫ اﺳﺗﻧﺎد  ﻣﺻوﺑﮫ  ﺟﻠﺳﮫ  1053  ﺗﺎرﯾﺦ  04/12/1403  ﻣوﺿوع  ﻋﻧﺎوﯾن  دروس  آزﻣون  ﻧﮭﺎﯾﯽ  دوره  دوم  \nﻣﺗوﺳطﮫ\n \nو\n \nﭼﮕوﻧﮕﯽ\n \nﺗرﻣﯾم\n \nﺳﺎﺑﻘﮫ\n \nﺗﺣﺻﯾﻠﯽ\n \nﺳواﺑق\n \nﺗﺣﺻﯾﻠﯽ،\n \nﺷﯾوه ﻧﺎﻣﮫ\n \nاﺟراﯾﯽ\n \nﻧﺣوه\n \nاﯾﺟﺎد\n \nﺳﺎﺑﻘﮫ\n \nﺗﺣﺻﯾﻠﯽ\n \nو\n \nﺗرﻣﯾم\n \nﺳﺎﺑﻘﮫ\n \nﺗﺣﺻﯾﻠﯽ\n \nآزﻣون ھﺎی\n \nﻧﮭﺎﯾﯽ\n \nدوره\n \nدوم\n \nﻣﺗوﺳطﮫ)اﺻﻼﺣﯾﮫ\n \nﺷﯾوه ﻧﺎﻣﮫ\n \nﻣﺻوب\n \nﭘﻧﺟﺎه\n \nو\n \nھﻔﺗﻣﯾن\n \nﺟﻠﺳﮫ\n \nﮐﻣﯾﺳﯾون\n \nﻣﻌﯾن\n \nﺷورای\n \nﻋﺎﻟﯽ\n \nآﻣوزش\n \nو\n \nﭘرورش\n \nﺗﺎرﯾﺦ\n \n27/09/1402\n(و\n \nﻣﺻوﺑﮫ\n \nﺟﻠﺳﮫ\n \n915\n \nﺷورای\n \nﻋﺎﻟﯽ\n \nاﻧﻘﻼب\n \nﻓرھﻧﮕﯽ\n \nﺗﺎرﯾﺦ.......\n \nﺑﮫ\n \nﺷرح\n \nزﯾر\n \nﺑﮫ\n \nﺗﺻوﯾب\n \nرﺳﯾد.\n ﺗﻌﺎرﯾف(اﻟف ﺳﺎﺑﻘﮫ ﺗﺣﺻﯾﻠﯽ:  ﻣطﺎﺑق  ﻣﺎده١  ﻣﺻوﺑﮫ  ﺟﻠﺳﮫ  ٣۴٨  ﺗﺎرﯾﺦ  15/04/1400  ﺷورای  ﻋﺎﻟﯽ  اﻧﻘﻼب  ﻓرھﻧﮕﯽ،  \nﺳﺎ

In [12]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(text_documents, embedding=embeddings)

/home/roshan/.virtualenvs/llm/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [ ]:
query = "نمونه سوال"
retriever = vectorstore.as_retriever()
retriever.invoke(query)


[Document(page_content='ﺑﺎﺳﻣﮫ ﺗﻌﺎﻟﯽ    اﺻﻼح ﻧﺎﻣﮫ ﺷﯾوه اﺟراﯾﯽ  ﻧﺣوه  اﯾﺟﺎد  و  ﺗرﻣﯾم  ﺳﺎﺑﻘﮫ  ﺗﺣﺻﯾﻠﯽ  ھﺎی آزﻣون  ﻧﮭﺎﯾﯽ  دوره  دوم  ﻣﺗوﺳطﮫ  ﻣﺻوب  \nﭘﻧﺟﺎه وھﻔﺗﻣﯾن)\n57\n(\n \nﺟﻠﺳﮫ\n \nﮐﻣﯾﺳﯾون\n \nﻣﻌﯾن\n \nﺷورای ﻋﺎﻟﯽ\n \nآﻣوزش\n \nو\n \nﭘرورش\n \nﺗﺎرﯾﺦ\n \n27/9/1402\n  ﺑﮫ اﺳﺗﻧﺎد  ﻣﺻوﺑﮫ  ﺟﻠﺳﮫ  1053  ﺗﺎرﯾﺦ  04/12/1403  ﻣوﺿوع  ﻋﻧﺎوﯾن  دروس  آزﻣون  ﻧﮭﺎﯾﯽ  دوره  دوم  \nﻣﺗوﺳطﮫ\n \nو\n \nﭼﮕوﻧﮕﯽ\n \nﺗرﻣﯾم\n \nﺳﺎﺑﻘﮫ\n \nﺗﺣﺻﯾﻠﯽ\n \nﺳواﺑق\n \nﺗﺣﺻﯾﻠﯽ،\n \nﺷﯾوه ﻧﺎﻣﮫ\n \nاﺟراﯾﯽ\n \nﻧﺣوه\n \nاﯾﺟﺎد\n \nﺳﺎﺑﻘﮫ\n \nﺗﺣﺻﯾﻠﯽ\n \nو\n \nﺗرﻣﯾم\n \nﺳﺎﺑﻘﮫ\n \nﺗﺣﺻﯾﻠﯽ\n \nآزﻣون ھﺎی\n \nﻧﮭﺎﯾﯽ\n \nدوره\n \nدوم\n \nﻣﺗوﺳطﮫ)اﺻﻼﺣﯾﮫ\n \nﺷﯾوه ﻧﺎﻣﮫ\n \nﻣﺻوب\n \nﭘﻧﺟﺎه\n \nو\n \nھﻔﺗﻣﯾن\n \nﺟﻠﺳﮫ\n \nﮐﻣﯾﺳﯾون\n \nﻣﻌﯾن\n \nﺷورای\n \nﻋﺎﻟﯽ\n \nآﻣوزش\n \nو\n \nﭘرورش\n \nﺗﺎرﯾﺦ\n \n27/09/1402\n(و\n \nﻣﺻوﺑﮫ\n \nﺟﻠﺳﮫ\n \n915\n \nﺷورای\n \nﻋﺎﻟﯽ\n \nاﻧﻘﻼب\n \nﻓرھﻧﮕﯽ\n \nﺗﺎرﯾﺦ.......\n \nﺑﮫ\n \nﺷرح\n \nزﯾر\n \nﺑﮫ\n \nﺗﺻوﯾب\n \nرﺳﯾد.\n ﺗﻌﺎرﯾف(اﻟف ﺳﺎﺑﻘﮫ ﺗﺣﺻﯾﻠﯽ:  ﻣطﺎﺑق  ﻣﺎده١  ﻣﺻوﺑﮫ  ﺟﻠﺳﮫ  ٣۴٨  ﺗﺎرﯾﺦ  15/04/1400  ﺷورای  ﻋﺎﻟﯽ  اﻧﻘﻼب  ﻓرھﻧﮕﯽ،  \nﺳﺎ

In [ ]:
retrieved_context = retriever.invoke(query)

In [ ]:
questions = [
    query
]

for question in questions:
    retrieved_context = retriever.invoke(question)
    formatted_prompt = prompt.format(context=retrieved_context, question=question)
    response_from_model = apply_chat_template_and_response(formatted_prompt)
    parsed_response = parser.parse(response_from_model)

    print(f"Question: {question}")
    print(f"Answer: {parsed_response}")
    print()

Question: نمرات پایه یازدهم چه تاثیری داره؟
Answer: در صورت انتخاب گروه آزمایشی غیر متناظر با رتبه تحصیلی خود، دانش آموزان پایه یازدهم می توانند در آزمون های پایه یازدهم و دوازدهم شرکت کنند. 


Question: چجوری ترمیم ثبت نام کنم؟
Answer: متاسفانه اطلاعاتی در مورد ترمیم ثبت نام در این متن وجود ندارد. 


Question: تو کی هستی؟
Answer: من یک دستیار هوش مصنوعی هستم. 




In [ ]:
while True:
    print("Say 'exit' or 'quit' to exit the loop")
    question = input('User question: ')
    print(f"Question: {question}")
    if question.lower() in ["exit", "quit"]:
        print("Exiting the conversation. Goodbye!")
        break
    formatted_prompt = prompt.format(context=retrieved_context, question=question)
    response_from_model = apply_chat_template_and_response(formatted_prompt)
    parsed_response = parser.parse(response_from_model)
    print(f"Answer: {parsed_response}")
    print()

Say 'exit' or 'quit' to exit the loop
Question: سلام
Answer: سلام 


Say 'exit' or 'quit' to exit the loop
Question: تو کی هستی؟
Answer: من یک دستیار هوش مصنوعی هستم. 


Say 'exit' or 'quit' to exit the loop
Question: چه سوالاتی میتونم ارت بپرسم؟
